In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.algo.discovery.dfg import algorithm as dfg_factory
from pm4py.visualization.dfg import visualizer as dfg_vis

# Load the event log
df = pd.read_csv('/BPIC2017_extended_log_generator_Sample.csv')
df = dataframe_utils.convert_timestamp_columns_in_df(df)
df['time:timestamp'] = pd.to_datetime(df['timestamp'])

# Define levels
activity_hierarchy = {
    "W_Complete application": {
        0: "General",
        1: "W_Complete application-Level1",
        2: "W_Complete application-Level2"
    },
    "W_Validate application": {
        0: "General",
        1: "W_Validate application-Level1"
    }
}

# UI setup
selected_levels_dict = {}
activity_boxes = []
output = widgets.Output()

# Frequency percentile slider
freq_slider = widgets.IntSlider(
    value=100,
    min=1,
    max=100,
    step=1,
    description='Top % Paths:',
    continuous_update=False,
    layout=widgets.Layout(width='50%')
)

# Create a slider box for each activity
for activity, levels in activity_hierarchy.items():
    max_level = max(levels.keys())
    default_val = 0

    slider = widgets.IntSlider(
        value=default_val,
        min=0,
        max=max_level,
        step=1,
        description='Level:',
        continuous_update=False,
        layout=widgets.Layout(width='90%')
    )

    selected_levels_dict[activity] = default_val

    def on_slider_change(change, activity=activity):
        if change['type'] == 'change' and change['name'] == 'value':
            selected_levels_dict[activity] = change['new']

    slider.observe(on_slider_change)

    box = widgets.VBox([
        widgets.HTML(f"<b>{activity}</b>"),
        slider
    ], layout=widgets.Layout(
        border='1px solid gray',
        padding='10px',
        margin='5px',
        width='45%'
    ))

    activity_boxes.append(box)

# Generate button
generate_button = widgets.Button(description="Generate Process Map")
generate_button.style.button_color = "#2ecc71"
generate_button.layout.width = '250px'

def on_generate_click(b):
    with output:
        clear_output()

        activity_level_map = {
            activity: activity_hierarchy[activity][selected_levels_dict[activity]]
            for activity in selected_levels_dict
        }

        temp_df = df.copy()
        temp_df['specialized_activity'] = temp_df['activity']

        for activity, level in activity_level_map.items():
            if level != "General" and level in temp_df.columns:
                mask = temp_df['activity'] == activity
                temp_df.loc[mask, 'specialized_activity'] = (
                    temp_df.loc[mask, 'activity'] + "_" + temp_df.loc[mask, level].fillna("").astype(str)
                )

        event_log = temp_df.rename(columns={
            'case_id': 'case:concept:name',
            'specialized_activity': 'concept:name',
            'timestamp': 'time:timestamp'
        })

        from pm4py.statistics.start_activities.log import get as start_activities_get
        from pm4py.statistics.end_activities.log import get as end_activities_get

        log = log_converter.apply(event_log, variant=log_converter.Variants.TO_EVENT_LOG)
        dfg = dfg_factory.apply(log)

        # Apply reversed frequency filtering based on top % paths
        percentile = freq_slider.value  # 1–100
        values = sorted(dfg.values(), reverse=True)

        cutoff_index = int(len(values) * (percentile / 100.0))
        cutoff_index = max(1, min(cutoff_index, len(values))) 
        cutoff_value = values[cutoff_index - 1] if values else 0

        filtered_dfg = {k: v for k, v in dfg.items() if v >= cutoff_value}

        start_activities = start_activities_get.get_start_activities(log)
        end_activities = end_activities_get.get_end_activities(log)

        gviz = dfg_vis.apply(
            filtered_dfg,
            log=log,
            variant=dfg_vis.Variants.FREQUENCY,
            parameters={
                "start_activities": start_activities,
                "end_activities": end_activities,
                "rankdir": "LR"
            }
        )
        dfg_vis.view(gviz)

generate_button.on_click(on_generate_click)

# UI layout
title = widgets.HTML("<h3 style='color:#2c3e50;'>CARPI Process Map Generator</h3>")
instructions = widgets.HTML("<p>Select specialization level for each activity below and filter paths by frequency (Disco-style):</p>")

activity_grid = widgets.HBox(activity_boxes)

controls_box = widgets.VBox([
    title,
    instructions,
    activity_grid,
    freq_slider, 
    generate_button
], layout=widgets.Layout(border='1px solid #ccc', padding='15px', width='100%'))

# Display UI
display(controls_box, output)

Output()